<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Get-files-and-genres" data-toc-modified-id="Get-files-and-genres-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get files and genres</a></span></li><li><span><a href="#Getting-and-normalizing-features" data-toc-modified-id="Getting-and-normalizing-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Getting and normalizing features</a></span><ul class="toc-item"><li><span><a href="#Zero-crossing" data-toc-modified-id="Zero-crossing-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Zero crossing</a></span></li><li><span><a href="#Spectral-Centroid" data-toc-modified-id="Spectral-Centroid-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Spectral Centroid</a></span></li><li><span><a href="#Spectral-Rolloff" data-toc-modified-id="Spectral-Rolloff-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Spectral Rolloff</a></span></li><li><span><a href="#Mel-Frequency-Cepstral-Coefficients-(MFCC)" data-toc-modified-id="Mel-Frequency-Cepstral-Coefficients-(MFCC)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Mel-Frequency Cepstral Coefficients (MFCC)</a></span><ul class="toc-item"><li><span><a href="#Get-dummies" data-toc-modified-id="Get-dummies-3.4.1"><span class="toc-item-num">3.4.1&nbsp;&nbsp;</span>Get dummies</a></span></li></ul></li><li><span><a href="#Chroma-stft" data-toc-modified-id="Chroma-stft-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Chroma stft</a></span></li></ul></li></ul></div>

# Libraries

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# For iterating through files
import os
# For audio playing
import IPython.display as ipd
# For visualizations
import librosa.display

# Get files and genres 

In [2]:
# Get genres

directory = 'data'
genres = []
for entry in os.scandir(directory):
    # echar un ojo para evitar .DS_Store
    if entry.name != ".DS_Store":
        genres.append(entry.name)
print(genres)

['Supernatural', 'Beautiful-happy', 'Chase-action']


In [3]:
# Get a dictionary -> key=genre , value = list of songs

songs_dict2 = {}
for gen in genres:
    dir = directory + f"/{gen}"
    songs2 = []

    for entry in os.scandir(dir):
        if entry.name.endswith(".wav"):
            songs2.append(entry.name)
            songs_dict2.update( {f"{gen}" : songs2} )

print (songs_dict2)

{'Supernatural': ['ES_Chasing Self - Polar Nights.wav', 'ES_Quantum Mechanics - Silver Maple.wav', 'ES_Far Far Far - Bonnie Grace.wav', 'ES_Satellites - Ebb & Flod.wav', 'ES_Midnight Tivoli - Luella Gren.wav', 'ES_Morphing Faces - Silver Maple.wav', 'ES_Mars Landing - Yi Nantiro.wav'], 'Beautiful-happy': ['ES_A Place for Us - Howard Harper-Barnes.wav', 'ES_Zanzibar - Jones Meadow.wav', 'ES_Make Me a Bird - David Celeste.wav', 'ES_The Light from Within - Howard Harper-Barnes.wav', 'ES_By Daylight - Gavin Luke.wav', 'ES_The Promising - Howard Harper-Barnes.wav', 'ES_Serene Story 2 - August Wilhelmsson.wav'], 'Chase-action': ['ES_Impulse - Hampus Naeselius.wav', 'ES_Vatican Mystery - Trailer Worx.wav', 'ES_One Last Mission Left - Eoin Mantell.wav', 'ES_The Great Escape - Christoffer Moe Ditlevsen.wav', 'ES_Dungeon - Dream Cave.wav']}


In [4]:
#Creating DataFrame
df = pd.DataFrame(columns=['Genre', 'Title', 'Offset'])
offset_15 = [0,15,30,45,60]
for i in songs_dict2:
    sng = songs_dict2.get(i)
    for s in sng:
        if s != ".DS_Store":
            for o in offset_15:
                my_dict = {"Title":f"{s}","Genre":f"{i}", "Offset":int(f"{o}")}
                df = df.append(my_dict, ignore_index=True)
'''           
            df_prueba.append(my_dict, ignore_index=True)'''
    
'''df = df.append({"Title":f"{s}","genre":f"{i}"}, ignore_index=True)'''
df

,Genre,Title,Offset
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60
...,...,...,...
90,Chase-action,ES_Dungeon - Dream Cave.wav,0
91,Chase-action,ES_Dungeon - Dream Cave.wav,15
92,Chase-action,ES_Dungeon - Dream Cave.wav,30
93,Chase-action,ES_Dungeon - Dream Cave.wav,45


# Getting and normalizing features

## Zero crossing

In [5]:
%%time
zero_crossings = []

for index, row in df.iterrows():
    path = f"data/{row['Genre']}/{row['Title']}"
    #print (type(row['Offset']))
    y, sr = librosa.load(path, duration=15, offset=row['Offset'])
    
    no_norm = librosa.zero_crossings(y=y, pad=False)
    zr=sum(no_norm)
    #print(no_norm)
    #zr = np.linalg.norm(no_norm)
    #print(zr)
    zero_crossings.append(zr)   

CPU times: user 1min 38s, sys: 214 ms, total: 1min 38s
Wall time: 1min 39s


In [6]:
df['Zero_crossings'] = zero_crossings

df.head()

,Genre,Title,Offset,Zero_crossings
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0,29991
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15,11088
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30,9427
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45,10185
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60,10435


## Spectral Centroid

In [7]:
%%time
spectral_centroid = []

for index, row in df.iterrows():
    path = f"data/{row['Genre']}/{row['Title']}"
    #print (type(row['Offset']))
    y, sr = librosa.load(path, duration=15, offset=row['Offset'])
    
    no_norm = librosa.feature.spectral_centroid(y=y)
    #spc= np.mean(no_norm)
    #print(no_norm)
    #zr = np.linalg.norm(no_norm)
    #print(zr)
    spectral_centroid.append(zr)

df['Spectral_centroid'] = spectral_centroid
df.head()

CPU times: user 44.7 s, sys: 148 ms, total: 44.9 s
Wall time: 44.9 s


,Genre,Title,Offset,Zero_crossings,Spectral_centroid
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0,29991,21262
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15,11088,21262
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30,9427,21262
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45,10185,21262
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60,10435,21262


In [8]:
df.tail()

,Genre,Title,Offset,Zero_crossings,Spectral_centroid
90,Chase-action,ES_Dungeon - Dream Cave.wav,0,18100,21262
91,Chase-action,ES_Dungeon - Dream Cave.wav,15,17814,21262
92,Chase-action,ES_Dungeon - Dream Cave.wav,30,14602,21262
93,Chase-action,ES_Dungeon - Dream Cave.wav,45,17752,21262
94,Chase-action,ES_Dungeon - Dream Cave.wav,60,21262,21262


## Spectral Rolloff

In [9]:
%%time
spectral_centroid = []

for index, row in df.iterrows():
    path = f"data/{row['Genre']}/{row['Title']}"
    #print (type(row['Offset']))
    y, sr = librosa.load(path, duration=15, offset=row['Offset'])
    
    no_norm = librosa.feature.spectral_rolloff(y=y, sr=sr)
    spr= np.mean(no_norm)
    #print(no_norm)
    #zr = np.linalg.norm(no_norm)
    #print(zr)
    spectral_centroid.append(spr)

df['Spectral_rolloff'] = spectral_centroid
df.head()

CPU times: user 44.9 s, sys: 134 ms, total: 45.1 s
Wall time: 45.1 s


,Genre,Title,Offset,Zero_crossings,Spectral_centroid,Spectral_rolloff
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0,29991,21262,2815.416309
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15,11088,21262,1452.841215
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30,9427,21262,1955.271510
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45,10185,21262,2053.587582
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60,10435,21262,2243.403101


## Mel-Frequency Cepstral Coefficients (MFCC)

In [30]:
%%time
mfcc = []

for index, row in df.iterrows():
    path = f"data/{row['Genre']}/{row['Title']}"
    y, sr = librosa.load(path, duration=15, offset=row['Offset'])
    
    no_norm = librosa.feature.mfcc(y=y, sr=sr)
    
    lista = []
    for m in no_norm:
        lista.append(m)
    
    lista_media = []
    for l in lista:
        lista_media.append(np.mean(l))
    
    mfcc.append(lista_media)


df['MFCC'] = mfcc
df.head()

CPU times: user 1min 51s, sys: 11.2 s, total: 2min 2s
Wall time: 46.1 s


,Genre,Title,Offset,Zero_crossings,Spectral_centroid,Spectral_rolloff,MFCC,Chroma_stft
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0,29991,21262,2815.416309,"[-242.92456, 136.92274, -24.217478, 26.947386,...",0.448233
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15,11088,21262,1452.841215,"[-207.92056, 186.96709, 1.4406731, 23.951105, ...",0.432779
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30,9427,21262,1955.271510,"[-152.80089, 171.50963, 33.763004, 20.416653, ...",0.434082
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45,10185,21262,2053.587582,"[-132.59921, 166.27094, 24.16506, 25.068478, 2...",0.440931
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60,10435,21262,2243.403101,"[-128.22403, 158.95535, 18.546083, 30.4618, 3....",0.454668


In [35]:
for index,row in df.iterrows():
    df['MFCC1'] = row['MFCC'][0]
    df['MFCC2'] = row['MFCC'][1]
    df['MFCC3'] = row['MFCC'][2]
    df['MFCC4'] = row['MFCC'][3]
    df['MFCC5'] = row['MFCC'][4]
    df['MFCC6'] = row['MFCC'][5]
    df['MFCC7'] = row['MFCC'][6]
    df['MFCC8'] = row['MFCC'][7]
    df['MFCC9'] = row['MFCC'][8]
    df['MFCC10'] = row['MFCC'][9]
    df['MFCC11'] = row['MFCC'][10]
    df['MFCC12'] = row['MFCC'][11]
    df['MFCC13'] = row['MFCC'][12]
    df['MFCC14'] = row['MFCC'][13]
    df['MFCC15'] = row['MFCC'][14]
    df['MFCC16'] = row['MFCC'][15]
    df['MFCC17'] = row['MFCC'][16]
    df['MFCC18'] = row['MFCC'][17]
    df['MFCC19'] = row['MFCC'][18]
    df['MFCC20'] = row['MFCC'][19]

df = df.drop(['MFCC'], axis=1)
df.head()

,Genre,Title,Offset,Zero_crossings,Spectral_centroid,Spectral_rolloff,Chroma_stft,MFCC1,MFCC2,MFCC3,...,MFCC11,MFCC12,MFCC13,MFCC14,MFCC15,MFCC16,MFCC17,MFCC18,MFCC19,MFCC20
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0,29991,21262,2815.416309,0.448233,-51.112473,136.885864,-19.684452,...,-3.794672,5.096334,-1.106382,3.135958,-3.689894,2.081623,-4.664995,2.550413,-3.662742,1.103073
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15,11088,21262,1452.841215,0.432779,-51.112473,136.885864,-19.684452,...,-3.794672,5.096334,-1.106382,3.135958,-3.689894,2.081623,-4.664995,2.550413,-3.662742,1.103073
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30,9427,21262,1955.271510,0.434082,-51.112473,136.885864,-19.684452,...,-3.794672,5.096334,-1.106382,3.135958,-3.689894,2.081623,-4.664995,2.550413,-3.662742,1.103073
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45,10185,21262,2053.587582,0.440931,-51.112473,136.885864,-19.684452,...,-3.794672,5.096334,-1.106382,3.135958,-3.689894,2.081623,-4.664995,2.550413,-3.662742,1.103073
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60,10435,21262,2243.403101,0.454668,-51.112473,136.885864,-19.684452,...,-3.794672,5.096334,-1.106382,3.135958,-3.689894,2.081623,-4.664995,2.550413,-3.662742,1.103073


## Chroma stft

In [26]:
%%time
chroma = []

for index, row in df.iterrows():
    path = f"data/{row['Genre']}/{row['Title']}"
    #print (type(row['Offset']))
    y, sr = librosa.load(path, duration=15, offset=row['Offset'])
    
    no_norm = librosa.feature.chroma_stft(y=y, sr=sr)
    chrm= np.mean(no_norm)
    #print(no_norm)
    #zr = np.linalg.norm(no_norm)
    #print(zr)
    chroma.append(chrm)

df['Chroma_stft'] = chroma
df.head()

CPU times: user 1min 52s, sys: 14.1 s, total: 2min 6s
Wall time: 51.1 s


,Genre,Title,Offset,Zero_crossings,Spectral_centroid,Spectral_rolloff,MFCC,Chroma_stft
0,Supernatural,ES_Chasing Self - Polar Nights.wav,0,29991,21262,2815.416309,-7.042403,0.448233
1,Supernatural,ES_Chasing Self - Polar Nights.wav,15,11088,21262,1452.841215,-7.960815,0.432779
2,Supernatural,ES_Chasing Self - Polar Nights.wav,30,9427,21262,1955.271510,-0.162135,0.434082
3,Supernatural,ES_Chasing Self - Polar Nights.wav,45,10185,21262,2053.587582,-0.672578,0.440931
4,Supernatural,ES_Chasing Self - Polar Nights.wav,60,10435,21262,2243.403101,0.917484,0.454668


In [27]:
df.tail()

,Genre,Title,Offset,Zero_crossings,Spectral_centroid,Spectral_rolloff,MFCC,Chroma_stft
90,Chase-action,ES_Dungeon - Dream Cave.wav,0,18100,21262,2661.250574,1.138421,0.493290
91,Chase-action,ES_Dungeon - Dream Cave.wav,15,17814,21262,2620.117490,3.511140,0.470293
92,Chase-action,ES_Dungeon - Dream Cave.wav,30,14602,21262,2470.768393,1.475577,0.431357
93,Chase-action,ES_Dungeon - Dream Cave.wav,45,17752,21262,2911.065731,1.163829,0.412305
94,Chase-action,ES_Dungeon - Dream Cave.wav,60,21262,21262,3043.048266,1.103073,0.388512
